In [1]:
import os

In [2]:
%pwd

'd:\\ERP_Sales_Forecasting\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ERP_Sales_Forecasting'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_file: Path
    result_file: Path

In [7]:
from ERPsalesForecasting.constants import *
from ERPsalesForecasting.utils.common import read_yaml, create_directories
from ERPsalesForecasting import logger

In [8]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, param_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.param = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])
        


    def get_model_training_config(self):
        config = self.config.model_training
        validationConfig = self.config.model_validation

        create_directories([config.root_dir, validationConfig.root_dir])
        column_names = "ModelName,MSE,MAE,R2\n"
        if (not os.path.exists(validationConfig.result_file) or os.path.getsize(validationConfig.result_file) == 0):
         with open(validationConfig.result_file, "w") as f:
            f.write(column_names)
            logger.info(f"Creating empty file: {validationConfig.result_file}")
            
        else:
            logger.info(f"{validationConfig.result_file} is already exists")

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            data_file=config.data_file,
            result_file=validationConfig.result_file,
        )

        return model_training_config

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np

In [10]:
from ERPsalesForecasting import logger

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [12]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config = config

    def data_preprocessing(self):

        df = pd.read_csv(self.config.data_file)

        encoder = OneHotEncoder(sparse=False)
        product_encoded = encoder.fit_transform(df[['ProductID']])
        product_encoded_df = pd.DataFrame(product_encoded, columns=[
                                          f'Product_{i}' for i in range(product_encoded.shape[1])])

        numeric_features = df.select_dtypes(
            include=[np.number]).drop(['SelledQTY'], axis=1)
        numeric_features.drop(['ProductTotalQty'], axis=1, inplace=True)
        return (product_encoded_df, numeric_features, df)

    def splitData(self, numeric_features, product_encoded_df, df):
        X = pd.concat([numeric_features, product_encoded_df], axis=1)
        y = df['SelledQTY']

        dates = df['Date']
        product_ids = df['ProductID']

        X_train, X_test, y_train, y_test, dates_train, dates_test, product_ids_train, product_ids_test = train_test_split(
            X, y, dates, product_ids, test_size=0.2, random_state=42)
        
        return (X_train, X_test, y_train, y_test, dates_train, dates_test, product_ids_train, product_ids_test)

    def dataScaling(self, X_train, X_test):
        # Normalize the Features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled
    
    def get_regression_model(model_type='linear'):
        if model_type == 'random_forest':
            return RandomForestRegressor(n_estimators=100, random_state=42)
        elif model_type == 'gradient_boosting':
            return GradientBoostingRegressor(n_estimators=100, random_state=42)
        # elif model_type == 'xgboost':
        #     return xgb(n_estimators=100, random_state=42)
        elif model_type == 'lasso':
            return Lasso(alpha=1.0, random_state=42)
        elif model_type == 'ridge':
            return Ridge(alpha=1.0, random_state=42)
        elif model_type == 'elastic_net':
            return ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
        elif model_type == 'decision_tree':
            return DecisionTreeRegressor(random_state=42)
        elif model_type == 'knn':
            return KNeighborsRegressor(n_neighbors=5)
        elif model_type == 'svr':
            return SVR()
        else:
            from sklearn.linear_model import LinearRegression
            return LinearRegression()
    
    def model_evaluation(self, model, model_name, X_test_scaled, y_test, dates_test, product_ids_test):
        y_pred = model.predict(X_test_scaled)
        
        # print(y_pred)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f'Model: {model_name}, MSE: {mse}, MAE: {mae}, R2 Score: {r2}')
        
        logger.info('Model: {model_name}, MSE: {mse}, MAE: {mae}, R2 Score: {r2}')
        
        # Your code for updating the DataFrame
        new_data = pd.DataFrame({
            'ModelName': ["Linear Regression"],
            'MSE': [mse],
            'MAE': [mae],
            'R2': [r2]
        })
        
        result_df = pd.read_csv(self.config.result_file)
        result_df = pd.concat([result_df, new_data], ignore_index=True) 
        
        logger.info('Model: {model_name}, evaluation data has been updated to results file')
    
        comparison_df = pd.DataFrame({
            'Date': dates_test,
            'ProductId': product_ids_test,
            'ActualSales': y_test,
            'PredictedSales': y_pred
        })
        
        comparison_df.sort_values(by=['Date', 'ProductId'], inplace=True)
        
        logger.info(f'{comparison_df.head()}')
        
        comparison_df.to_csv(f'{self.config.root_dir}/{model_name}.csv')
        
        logger.info(f'test result of {model_name} is saved to: {self.config.root_dir}/{model_name}.csv')

In [13]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config)
    product_encoded_df, numeric_features, df = model_training.data_preprocessing()
    X_train, X_test, y_train, y_test, dates_train, dates_test, product_ids_train, product_ids_test = model_training.splitData(product_encoded_df, numeric_features, df)
    X_train_scaled, X_test_scaled = model_training.dataScaling(X_train, X_test)
    
    model = model_training.get_regression_model()
    model.fit(X_train_scaled, y_train)
    
    model_training.model_evaluation(model, "Linear Regression", X_test_scaled, y_test, dates_test, product_ids_test)
    
    
except Exception as e:
    raise e

[2023-12-21 21:45:37,921]: INFO: common : 31: yaml file: config\config.yaml loaded successfully
[2023-12-21 21:45:37,923]: INFO: common : 31: yaml file: params.yaml loaded successfully
[2023-12-21 21:45:37,923]: INFO: common : 50: created directory at: artifacts
[2023-12-21 21:45:37,924]: INFO: common : 50: created directory at: artifacts/model_training
[2023-12-21 21:45:37,924]: INFO: common : 50: created directory at: artifacts/model_validation
[2023-12-21 21:45:37,926]: INFO: 3819670936 : 19: Creating empty file: artifacts/model_validation/result.csv
Model: Linear Regression, MSE: 900.0, MAE: 30.0, R2 Score: nan
[2023-12-21 21:45:37,938]: INFO: 2587357830 : 69: Model: {model_name}, MSE: {mse}, MAE: {mae}, R2 Score: {r2}
[2023-12-21 21:45:37,941]: INFO: 2587357830 : 82: Model: {model_name}, evaluation data has been updated to results file
[2023-12-21 21:45:37,945]: INFO: 2587357830 : 93:          Date  ProductId  ActualSales  PredictedSales
1  2023-12-18       5386           30      

d:\ERP_Sales_Forecasting\env\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
d:\ERP_Sales_Forecasting\env\Lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\priya\AppData\Local\Temp\ipykernel_26408\2587357830.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_data], ignore_index=True)
